# Machine Learning Workshop - Exploring small molecule potency

## Outline
- Task Overview
- Data Exploring
- Supervised Learning
    - ~~Classification (Less-potent vs Potent PPI inhibitors)~~ (Lecture 1 - Alex de Sá)
    - Regression (PPI inhibitory potency)
        - Decision Tree
        - Performance Evaluation
        - Random Forest - Ensemble methods

### Task Overview
The problem we will solve is **predicting inhibitory activity of small molecules against certain protein-protein interactions (PPIs)**. 

Proteins rarely act alone as their functions tend to be regulated. Many molecular processes within a cell are dependents of PPIs. Some PPIs are involved in multiple aggregation-related diseases, such as Creutzfeldt–Jakob and Alzheimer's diseases. The discovery of novel molecules capable of inhibiting these processes can be of great importance for medicine.

Small molecules are low molecular weight molecules that include lipids, monosaccharides, second messengers, other natural products and metabolites, as well as drugs and other xenobiotics. They can interact with receptors and regulate biological processes. 

The first thing we need is a data set with inhibitory activity values for real molecules. For this workshop, we will use datasets from [TIMBAL](https://pubmed.ncbi.nlm.nih.gov/23766369/) and [iPPIDB](https://pubmed.ncbi.nlm.nih.gov/33416858/). 

The inhibitory activity values are reported in **-log(IC50)**. IC50 means how much of a particular inhibitory drug is needed to inhibit a given biological process or biological component by 50\%. It is measured in uM (micromolar).


### Loading libraries

In [ ]:
%load_ext autoreload
%autoreload 2

from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
from rdkit.Chem.Draw import IPythonConsole #RDKit drawing
# A few settings to improve the quality of structures 
from rdkit.Chem import rdDepictor
IPythonConsole.ipython_useSVG = True
rdDepictor.SetPreferCoordGen(True)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()

from math import sqrt
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import Chem
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, ConfusionMatrixDisplay
from scipy.stats import pearsonr, kendalltau, spearmanr
from tabulate import tabulate

import numpy as np
import pandas as pd
import time
import statsmodels.api as sm

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

### Data Exploration

In [ ]:
!wget https://raw.githubusercontent.com/carlosmr12/mlwk2023/master/data/ppi_inhibitors.csv

In [ ]:
df_data = pd.read_csv("ppi_inhibitors.csv") # Load data from file to a DataFrame structure
print(df_data.shape) # .shape displays how the dataframe (matrix) looks like
df_data.head() # .head() displays the first few items in the dataframe

Data distribution based on data sources

In [ ]:
print(df_data['database'].value_counts())

fig = px.pie(df_data, values=df_data['database'].value_counts().values, \
             names=df_data['database'].value_counts().index, \
             title='Data sources')

fig.show(renderer="colab")

The molecules of the dataset are represented through **SMILES**. SMILES is a chemical notation that allows representation of a chemical structure.  They can be represented using simple vocabulary (atom and bond symbols), and few grammar rules: 

- ***2-Propanol would be “CC(O)C”***

- ***2-Methylbutanal would be “CC(C)CC(=O)”.***

Using a type of SMILES called ISOMERIC SMILES it is even possible to represent specific isotopism, configuration about double bonds, and chirality. 

Below you can see some examples of SMILES in our data:


2D depiction of small molecules based on the SMILES format

In [ ]:
print(df_data.iloc[0]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[0]['SMILES'])
m

In [ ]:
print(df_data.iloc[-1]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[-1]['SMILES'])
m

In [ ]:
print(df_data.iloc[1]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[2]['SMILES'])
m

What does the distribution of our target variable look like?

In [ ]:
target = "Experimental IC50 (log10)"
fig = px.histogram(df_data, x=target)
fig.show(renderer="colab")

Why do we would preferably work with the target variable in a log scale?

In [ ]:
df_data['inverse_log'] = 10**df_data[target]
fig = px.histogram(df_data, x='inverse_log')
fig.show(renderer="colab")

It is also important to note that ours datasets covers, besides a long range of concentrations, also a range of values for diferent properties. This is also important to generate more generalised  models.

An important concept here is **feature**. A feature is a property of the object you’re trying to predict. It can also be referred to as indepent variable, since it is a fixed property of the data point and it does not depend of others variables. These independent variables are essential, because the algorithms need characteristics of the data points as support for the learning process and predicting the labels (in our case inhibitory activity).

In [ ]:
columns = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
           "RingCount", "MolWt"]

fig = make_subplots(rows=3, cols=2, start_cell="bottom-left")

nlines = 3
ncolumns = 2

for i in range(nlines):
    for j in range(ncolumns):
        fig.add_trace(go.Histogram(x=df_data[columns[i+j]], name=columns[i+j]), row=i+1, col=j+1)

fig.show(renderer="colab")

### Regression (PPI inhibitory potency)

#### Data Preparation

The method **train_test_split()** used in the code below divides our data into two subsets: 

- One subset to *train* the model
- A subset to evaluate or *test* how good your model is, which should
    - Be large enough to yield statistically meaningful results
    - Be representative of the data set as whole

*(Never train on test data)*

Splitting your dataset into training and test sets is very important and it is directly related to your models ability to *learn* patterns during the training step, which will ideally help to make it generalisable. Using the test set (a subset completely unseen during trainning) you can estimate the performance of the model when it is applied to new data points.

In [ ]:
target = "Experimental IC50 (log10)"

# Basic properties
basic_properties = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
                    "RingCount", "MolWt"]

# What is the best TEST_SIZE?
TEST_SIZE = 0.2

X = df_data[basic_properties]
y = df_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                    random_state=42)

X_train_fpr = df_data.iloc[X_train.index].drop(["Experimental IC50 (log10)", "SMILES", "database"], axis=1)
X_test_fpr = df_data.iloc[X_test.index].drop(["Experimental IC50 (log10)", "SMILES", "database"], axis=1)
y_train_fpr = df_data.iloc[X_train.index][target]
y_test_fpr = df_data.iloc[X_test.index][target]

fig = make_subplots(rows=1, cols=2, start_cell="bottom-left")

fig.add_trace(go.Histogram(x=y_train, name='Training set'), row=1, col=1)
fig.add_trace(go.Histogram(x=y_test, name='Test set'), row=1, col=2)

fig.show(renderer="colab")

#### Decision Tree

In [ ]:
regressor = DecisionTreeRegressor(random_state=42, max_depth=3)
y_pred_train = cross_val_predict(regressor, X_train, y_train, cv=5)

regressor.fit(X_train, y_train)
y_pred_test = regressor.predict(X_test)

rmse_train = round(sqrt(mean_squared_error(y_train, y_pred_train)), 2)
pearsons_train = round(pearsonr(y_train, y_pred_train)[0], 2)
kendalls_train = round(kendalltau(y_train, y_pred_train)[0], 2)
spearmans_train = round(spearmanr(y_train, y_pred_train)[0], 2)
rmse_test = round(sqrt(mean_squared_error(y_test, y_pred_test)), 2)
pearsons_test = round(pearsonr(y_test, y_pred_test)[0], 2)
kendalls_test = round(kendalltau(y_test, y_pred_test)[0], 2)
spearmans_test = round(spearmanr(y_test, y_pred_test)[0], 2)

d = [ ["RMSE", rmse_train, rmse_test],
     ["Pearson's", pearsons_train, pearsons_test],
     ["Kendall's", kendalls_train, kendalls_test],
     ["Spearman's", spearmans_train, spearmans_test]]

print(tabulate(d, headers=["Metric", "Training", "Test"]))

In [ ]:
fig = make_subplots(rows=1, cols=2, start_cell="bottom-left")

fig.add_trace(go.Scatter(x=y_train, y=y_pred_train, name='Training set', 
                         mode="markers"), row=1, col=1)
fig.add_trace(go.Scatter(x=y_test, y=y_pred_test, name='Test set', 
                         mode="markers"), row=1, col=2)

x = sm.add_constant(y_train)
p = sm.OLS(y_pred_train, x).fit().params
x = np.arange(y_train.min(), y_train.max())
y = p.const + p[target] * x
fig.add_trace(go.Scatter(x=x, y=y, name='', mode="lines", 
                         line=dict(dash='dash', color="black"), 
                         showlegend=False), row=1, col=1)

x = sm.add_constant(y_test)
p = sm.OLS(y_pred_test, x).fit().params
x = np.arange(y_test.min(), y_test.max())
y = p.const + p[target] * x
fig.add_trace(go.Scatter(x=x, y=y, name='', mode="lines", 
                         line=dict(dash='dash', color="black"), 
                         showlegend=False), row=1, col=2)
fig.show(renderer="colab")

Which feature is more important for the model?

In [ ]:
labels = []
scores = []
try:
    for feature,score in zip(regressor.feature_names_in_, regressor.feature_importances_):
        if score != 0:
            labels.append(feature)
            scores.append(round(score,2))
except AttributeError as e:
    for feature,score in zip(range(0, len(regressor.feature_importances_)), regressor.feature_importances_):
        if score != 0:
            labels.append(feature)
            scores.append(round(score,2))
fig = px.bar(x=labels, y=scores, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")

In [ ]:
fig, axes = plt.subplots(nrows=1 ,ncols=1, figsize=(9,6), dpi=300)
tree.plot_tree(regressor, feature_names=X_train.columns.tolist(), filled=True)
plt.show()

#### Performance Evaluation Metrics

#### Random Forest